In [3]:
class Node:
    """
        Set up Node class to represent vertices of a Graph, each node 
        contains its vertex value and edges.
    """
    def __init__(self, vertex):
        self.vertex = vertex
        self.edges = {}
        
    def add_edge(self, vertex, count):
        if vertex != self.vertex:
            self.edges[vertex] = self.edges.get(vertex, 0) + count
            
    def update_edge(self, from_vertex, to_vertex):
        """
            Update the edges to change from from_node's vertex to to_node's vertex
            If to_node is already existed in edges, combine their counts
        """
        if from_vertex in self.edges.keys():
            self.edges[to_vertex] = self.edges.get(to_vertex,0) + self.edges[from_vertex]
            del self.edges[from_vertex]
                       
    def copy(self):
        node = Node(self.vertex)
        node.edges = dict(self.edges)
        return node
    
    def edge_size(self):
        return len(self.edges)
            
    def __str__(self):
        return f"Vertex {self.vertex}: {self.edges}"
    
        
class Graph:
    """
        Set up a Graph class to represents all vertice and their edges in a form of dictionary:
        For example:
        {
            1: {1:{2:1, 3:1}}
            2: {2:{1:1, 3:1, 4:1}}
            3: {3:{1:1, 2:1, 4:1}}
            4: {4:{2:1, 3:1}}   
        }
    """
    def __init__(self, nodes):
        self.nodes = {node.vertex: node.copy() for node in nodes}
        
    def copy(self):
        # to create a copy of the Graph in order to perform minimum cuts without changing original Graph
        nodes = []
        for node in self.nodes.values():
            nodes.append(node)
        copy_graph = Graph(nodes)
        return copy_graph
            
    def delete_node(self, vertex):
        del self.nodes[vertex]
        
    def merge(self, vertex1, vertex2):
        """
            Merge/ contract current node1 and node2. vertex is still node1's vertex,
            combine edges of two nodes, excluding edges having vertex of node1 and node2.
            For example: two nodes chosen is vertex 1 and vertex 3 of the Graph
                         Vertex 1: {2:1, 3:1}
                         Vertex 3: {1:1, 2:1, 4:1}
                         merge(vertex1, vertex3) 
                         >>> Vertex 1: {2:2, 4:1}
        """
        # remove node 2 from node1's edges
        if vertex2 in self.nodes[vertex1].edges.keys():
            del self.nodes[vertex1].edges[vertex2]
        
        #Add key and value of node2's edges only if the key doesn't equal to node1's vertex
        for key, value in self.nodes[vertex2].edges.items():
            if key != vertex1:
                self.nodes[vertex1].edges[key] = self.nodes[vertex1].edges.get(key, 0) + value 
    
    def contraction(self, vertex1, vertex2):
        """
            Contraction for random contraction algorithm
        """
        self.merge(vertex1, vertex2)   # merge node1 and node2 to node1
        self.delete_node(vertex2)        #delete node2
        for node in self.nodes.values():       # update all other edges to replace node2 with node1
            node.update_edge(vertex2, vertex1)  
            
    def random_contraction(self):
        """
            Random contraction algorithm
            For example:
            chosen vertex1 is 4, and vertex2 is 2. The result of random_contraction is:
            >>> Vertex 1: {3: 1, 4: 1}, Vertex 3: {1: 1, 4: 2}, Vertex 4: {3: 2, 1: 1}
        """
        import random
        node1 = random.choice(list(self.nodes.values()))
        vertex1 = node1.vertex
        vertex2 = random.choice(list(node1.edges.keys()))
        self.contraction(vertex1, vertex2)
        
    def minimum_cuts(self):
        """
            Use random contraction algorithm to calculate minimum cut.
            Reach minimum cut when there's only 2 vertices left, and the count is the minimum cut
            Note that there's possibility to fail to find the real minimu cut by only running this once
            Therefore, to find real minimum cut, should run at least n^2 time, where n is the count of
            vertex in the Graph
            After performing mimimum_cuts, the resulted Graph looks like this:
            >>> Vertex 1: {2: 3}, Vertex 2: {1: 3}
        """
        import random
        
        # Make a copy of the Graph
        graph = self.copy()
        
        # Repeatly perform random contraction when there're more than 2 vertices in the Graph
        while graph.size() > 2:
            graph.random_contraction()
            
        # To get the edge count as the minimum cut number
        node = list(graph.nodes.values())[0]
        edge_count = list(node.edges.values())[0]
        return edge_count
    
    def simulation(self, n):
        """
            To run minimum_cuts n times to find the real minimum cut
            For the example in Graph documentation, the result of simulation(10) could be
            >>> 2
        """
        results = []
        for i in range(n):
            results.append(self.minimum_cuts())
        return min(results)
            
    def size(self):
        return len(self.nodes) 
    
    def __str__(self):
        return ", ".join([str(node) for node in self.nodes.values()])

    
if __name__ == "__main__":
    
    # initiate the Nodes and the Graph
    line = [1, 2, 3] 
    line2 = [2, 1, 3, 4]
    line3 = [3, 1, 2, 4]
    line4 = [4, 2, 3]
    
    node = Node(line[0])
    node2 = Node(line2[0])
    node3 = Node(line3[0])
    node4 = Node(line4[0])
    
    for i in range(1, len(line)):
        node.add_edge(line[i], 1)

    for i in range(1, len(line2)):
        node2.add_edge(line2[i], 1)

    for i in range(1, len(line3)):
        node3.add_edge(line3[i], 1)

    for i in range(1, len(line4)):
        node4.add_edge(line4[i], 1)

    nodes = [node, node2, node3, node4]
    graph = Graph(nodes)
    print("the orinal Graph: \n", graph)
    print("\n")
    
    # calculation the minimum cut by simulating n^2 time (4*4), and make sure the Graph didn't change
    cut = graph.simulation(16)
    print("the Graph after performing minimum cut: \n", graph)
    print("the minimum cut is: ", cut)
        

the orinal Graph: 
 Vertex 1: {2: 1, 3: 1}, Vertex 2: {1: 1, 3: 1, 4: 1}, Vertex 3: {1: 1, 2: 1, 4: 1}, Vertex 4: {2: 1, 3: 1}


the Graph after performing minimum cut: 
 Vertex 1: {2: 1, 3: 1}, Vertex 2: {1: 1, 3: 1, 4: 1}, Vertex 3: {1: 1, 2: 1, 4: 1}, Vertex 4: {2: 1, 3: 1}
the minimum cut is:  2


In [13]:
def homework4():
    """HW4: https://www.coursera.org/learn/algorithms-divide-conquer/exam/AUmUg/programming-assignment-4/attempt
         The file contains the adjacency list rdepresentation of a simple undirected graph. There are 200 vertices 
         labeled 1 to 200. The first column in the file represents the vertex label, and the particular row (other 
         entries except the first column) tells all the vertices that the vertex is adjacent to. So for example, 
         the 6th row looks like : "6 155 56 52 120......". This just means that the vertex with label 6 is adjacent 
         to (i.e., shares an edge with) the vertices with labels 155,56,52,120,......,etc
         Your task is to code up and run the randomized contraction algorithm for the min cut problem and use it on 
         the above graph to compute the min cut.  
    """
    
    # read the input file, and output the adjacency list of the Graph
    try:
        with open("kargerMinCut.txt") as file:
            nodes = []
            lines = file.readlines()
            for line in lines:
                vertex = [int(i) for i in line.strip().split("\t")]
                node = Node(vertex[0])
                for i in range(1, len(vertex)):
                    node.add_edge(vertex[i], 1)
                nodes.append(node)    
            graph = Graph(nodes)
        
        # run n^2 (200*200, where n represents count of vertices) simulation to get the minimum cut of the Graph
        size = graph.size()    
        cut = graph.simulation(size**2)
        print("The minimum cut is: ", cut)
            
    except:
        print("Something went wrong")
        

if __name__ == "__main__":
    import time
    start = time.time()
    homework4()
    end = time.time()
    print(f"The run time of 40,000 simulation is {end - start} second(s)")

The minimum cut is:  17
The run time of 40,000 simulation is 316.2466015815735 second(s)
